**Imports:**

In [1]:
import os
import csv
import enum
from tqdm.notebook import tqdm
import numpy as np
import itertools
import datetime
from collections import namedtuple, OrderedDict

from pyxcel.sheet_manager import SheetManager
from utils.check_spar import check_spar

**DS Definitions:**

In [2]:
class SheetReference(enum.Enum):
    
    # inputs
    Aerofoil = ('Aerodynamics', 'V2')
    AR = ('Aerodynamics', 'C3')
    TaperRatio = ('Aerodynamics', 'C7')
    
    # outputs
    BatteryMass = ('Mass Breakdown', 'C33')
    MaxStress = ('Structures', 'L78')
    
    # other
    AerofoilRange = ('Aerofoil Data','A8')
    Chord = ('Stability', 'B9')
    SparWidth = ('Structures', 'A74')
    SparHeight = ('Structures', 'B74')

In [3]:
class CombinationReference(enum.IntEnum):
    
    Aerofoil = 0
    AR = 1
    TaperRatio = 2

In [4]:
SparDesign = namedtuple('SparDesign', ('h', 'b', 'ix'))

**Problem Setup:**

In [5]:
# set tolerance for spar height / width (absolute value in meters)
hb_tolerance = 0.005

In [6]:
sheet_path = 'DesignSpreadsheet.xlsx'
af_dir = os.path.join(os.getcwd(), 'aerofoil_dat')

In [7]:
workbook = SheetManager(sheet_path)

In [8]:
input_keys = (SheetReference.Aerofoil, SheetReference.AR, SheetReference.TaperRatio)
output_keys = (SheetReference.BatteryMass, SheetReference.MaxStress)

In [9]:
# check all input keys are included in CombinationReference
assert set(ref.name for ref in CombinationReference) == set(ref.name for ref in input_keys)

# check all input/output keys are defined
assert all(SheetReference[key.name] is not None for key in itertools.chain(input_keys, output_keys))

In [25]:
# aerofoils = list(filter(None, workbook.get_cell(*SheetReference.AerofoilRange.value).value))
aerofoils=['E472']
# need to increase resolution here
ARs = np.arange(5,10,0.125)
taper_ratios = np.arange(0.25,1.00001,0.025)

In [28]:
len(design_combinations)

1240

In [27]:
design_combinations = list(itertools.product(aerofoils, ARs, taper_ratios))

**Run Design Sweep:**

In [29]:
# define t_epoch to avoid overwriting file
t_epoch = int(datetime.datetime.now().timestamp())
csv_path = f'design_sweep_{t_epoch}.csv'

for idx, combination in enumerate(tqdm(design_combinations)):
    
    # setup dict
    design_params = OrderedDict()
    
    # change each cell
    for key in input_keys:
        workbook.change_cell(*key.value, combination[CombinationReference[key.name]])
        design_params[key.name] = combination[CombinationReference[key.name]]
        
    # load aerofoil
    af_path = os.path.join(af_dir, f'{combination[CombinationReference.Aerofoil]}.dat')
    
    try:
        af = np.loadtxt(af_path, skiprows=1, delimiter=',')
    except OSError:
        continue
    
    # get aerofoil chord length
    chord_length = workbook.check_cell(*SheetReference.Chord.value)
    
    b_range = np.arange(0.01,0.0400001,0.002)#consider multiples of material thickness
    h_range = np.arange(0.01,0.0400001,0.002)
    
    # get spar with max(ix)
    spar = SparDesign(0, 0, 0)
    for b, h in itertools.product(b_range, h_range):
        ix = b * h ** 3 / 12
        
        if check_spar(af, chord_length, h, b, h * hb_tolerance, b * hb_tolerance) and ix > spar.ix:
            spar = SparDesign(h, b, ix)
    
    if spar.h == 0 or spar.b== 0:
        design_params[SheetReference.SparHeight.name] = spar.h
        design_params[SheetReference.SparWidth.name] = spar.b
        for key in output_keys:
            design_params[key.name] = 'SPAR FAIL'
    else:
        # set spar values
        workbook.change_cell(*SheetReference.SparHeight.value, spar.h)
        design_params[SheetReference.SparHeight.name] = spar.h

        workbook.change_cell(*SheetReference.SparWidth.value, spar.b)
        design_params[SheetReference.SparWidth.name] = spar.b

        for key in output_keys:
            design_params[key.name] = workbook.check_cell(*key.value)
    
    # write to csv
    csv_exists = os.path.exists(csv_path)
    with open(csv_path, 'a+') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=design_params, delimiter=',', lineterminator='\n')
        
        if not csv_exists:
            writer.writeheader()
            
        writer.writerow(design_params)

OrderedDict([('Aerofoil', 'S7075 (B)'), ('AR', 4), ('TaperRatio', 0.45), ('SparHeight', 0.01), ('SparWidth', 0.01), ('BatteryMass', None), ('MaxStress', 7008009.356686389)])